In [4]:
import pandas as pd
import spacy as sp

tokens = pd.read_csv("../data/tokenized_data.csv")
nlp = sp.load("en_core_web_sm")

print("read")

read


In [ ]:
def atomic_sentence_extract(doc):
    atomic_sentences = []

    for sentence in doc.sents:
        #extract the subjects and objects and the main verb ROOT
        ##first get the subjects.

        subjects = []
        for tok in sentence.root.lefts:
            if tok.dep_ in ("nsubj", "nsubpass"):
                subjects.append(tok.text)
        if subjects.len() == 0:
            subjects = ["X"]

        #get the objects/complements
        objects = []
        for tok in sentence.root.right:
            if tok.dep_ in ("dobj", "attr", "prep", "xcomp"):
                objects.append(tok.text)
        if objects.len() == 0:
            objects = ["X"]

        #also want to separate relative clauses
        for tok in sentence:
            if tok.dep_ in ("relcl", "advcl", "ccomp"):
                relative_clause = " ".join([t.text for t in tok.subtree])
                atomic_sentences.append(relative_clause)

        #now we want to combine subjs/objs to create the other atomic sentences
        atomic = f"{subjects[0]} {sentence.root.lemma_} {' '.join(objects)}".strip()
        atomic_sentences.append(atomic)